In [77]:
%%capture 
!pip install -r requirements.txt

In [78]:
from enum import Enum, StrEnum

class Scaling(Enum):
    INDEPENDENT = 1
    JOINT = 2

class DatasetType(StrEnum):
    TRAIN = 'trn'
    TEST = 'tst'
    VALIDATION = 'trn'

# Global variable to enable debug mode
DEBUG = True

#### Data loading & pre-processing functions

In [79]:
import pandas as pd
import numpy as np
from sklearn import preprocessing


# *Helper function to preprocess the RSSI data
def preprocess_rssi_data(df_rssi: pd.DataFrame, scaling_strategy: Scaling) -> pd.DataFrame:
    """
    This function preprocesses the training data by:
    1. Replacing all 100 values with -110 (ensures continuity of data)
    2. Separating the RSS values from the labels
    3. Scaling the data to have zero mean and unit variance

    Parameters:
    - train: The training data to be preprocessed
    - scaling_strategy: The scaling strategy to be used (INDEPENDENT or JOINT)
    """
    
    # 1. replace all 100 values with -110 (ensures continuity of data)
    df = df_rssi.replace(100, -110)
    
    # 2. Separate the RSS values from the labels
    rssiValues = df.iloc[:, :-3]
    labels = df.iloc[:, -3:]
    
    # 3. Scale the data to have zero mean and unit variance
    # This is done either independently for each AP or jointly for all APs
    if scaling_strategy == Scaling.INDEPENDENT:
        scaler = preprocessing.StandardScaler()

        scaled_rss = scaler.fit_transform(rssiValues)
        df_scaled_rss = pd.DataFrame(scaled_rss, columns=rssiValues.columns)
        df = pd.concat([df_scaled_rss, labels], axis=1)
    
    elif scaling_strategy == Scaling.JOINT:
        flattened = rssiValues.values.flatten()
        global_mean = np.mean(flattened)
        global_std = np.std(flattened)
        
        scaled_rss = (rssiValues - global_mean) / global_std
        df = pd.concat([scaled_rss, labels], axis=1)
        df = df.reset_index(drop=True)
    
    else: 
        raise NotImplementedError("Specified scaling strategy is not implemented, use either Scaling.INDEPENDENT or Scaling.JOINT.")
    
    return df

# # *Load and pre-process the training data
# def get_preprocessed_training_data(data_path: str, training_months: list[str], num_APs: int, scaling_strategy: Scaling, floor: int) -> pd.DataFrame:
#     """
#     This function loads and preprocesses the training data from the specified training months and floor.

#     Parameters:
#     - data_path: The path to the data
#     - training_months: The list of training months to be used
#     - num_APs: The number of access points
#     - scaling_strategy: The scaling strategy to be used (INDEPENDENT or JOINT)
#     - floor: The floor to be used
#     """
#     # Since the csv files do not have column names, we define these first.
#     list_of_APs = ["AP" + str(i) for i in range(0, num_APs)]

#     # Load the training data from all specified training sets.  
#     df_rss = pd.concat([pd.read_csv(data_path + training_set + 'trn01rss.csv', names=list_of_APs) for training_set in training_months])
#     df_rss = df_rss.reset_index(drop=True)
    
#     # Get all x,y,floor labels (gotten from data_path + training_month + 'trn01crd.csv')
#     df_labels = pd.concat([pd.read_csv(data_path + training_set + 'trn01crd.csv', names=['x', 'y', 'floor']) for training_set in training_months])
#     df_labels = df_labels.reset_index(drop=True)

#     # Add the labels to the pre-processed data
#     df_labeled = pd.concat([df_rss, df_labels], axis=1)
    
#     # Filter the data to only include the specified floor
#     df_labeled = df_labeled[df_labeled['floor'] == floor]

#     # Pre-processing of the training data
#     df_train = preprocess_rssi_data(df_labeled, scaling_strategy)
    
#     return df_train

# *Load and pre-process the data
def get_preprocessed_dataset(data_path: str, months: list[str], sets: list[str], type: DatasetType, num_APs: int, scaling_strategy: Scaling, floor: int) -> pd.DataFrame:
    """
    This function loads and preprocesses the training data from the specified training months and floor.

    Parameters:
    - data_path: The path to the data
    - months: The list of months to be used
    - sets: The list of set numbers to be used
    - type: The type of dataset to be made (TRAIN, TEST or VALIDATION)
    - num_APs: The number of access points
    - scaling_strategy: The scaling strategy to be used (INDEPENDENT or JOINT)
    - floor: The floor to be used
    """
    # Since the csv files do not have column names, we define these first.
    list_of_APs = ["AP" + str(i) for i in range(0, num_APs)]

    # Load the test data from all specified test sets.  
    df_test_rss = pd.concat([pd.read_csv(data_path + month + '/' + type + set + 'rss.csv', names=list_of_APs) for month in months for set in sets])
    df_test_rss = df_test_rss.reset_index(drop=True)
    
    # Get all x,y,floor labels
    df_test_labels = pd.concat([pd.read_csv(data_path + month + '/' + type + set + 'crd.csv', names=['x', 'y', 'floor']) for month in months for set in sets])
    df_test_labels = df_test_labels.reset_index(drop=True)

    # Add the labels to the pre-processed data
    df_test_labeled = pd.concat([df_test_rss, df_test_labels], axis=1)
    
    # Filter the data to only include the specified floor
    df_test_labeled = df_test_labeled[df_test_labeled['floor'] == floor]

    # Pre-processing of the training data
    df_test = preprocess_rssi_data(df_test_labeled, scaling_strategy)
    
    return df_test
    

#### SETUP

In [80]:
data_path = './data/'
training_months = ['01', '02', '03', '04', '05']
sets = ['01']
type = DatasetType.TRAIN
num_APs = 620
scaling_strategy = Scaling.JOINT
floor = 3


df_train_full = get_preprocessed_dataset(data_path, training_months, sets, type, num_APs, scaling_strategy, floor)
df_train_x = df_train_full.iloc[:, :-3] # Just the RSSI values
df_train_y = df_train_full.iloc[:, -3:-1] # Just the x and y coordinates (no floor)

if DEBUG: print('df_train_full:', df_train_full.shape)


df_train_full: (1440, 623)


In [81]:
months = ['01', '02', '03', '04', '05']
sets = ['01'] # 01 Corresponds to the same locations as the training set
type = DatasetType.TEST

df_test_full = get_preprocessed_dataset(data_path, months, sets, type, num_APs, scaling_strategy, floor)
df_test_x = df_test_full.iloc[:, :-3] # Just the RSSI values
df_test_y = df_test_full.iloc[:, -3:-1] # Just the x and y coordinates (no floor)

if DEBUG: print('df_test_full:', df_test_full.shape)

df_test_full: (1440, 623)


In [82]:
months = ['01']
sets = ['02', '03', '04']
type = DatasetType.VALIDATION

df_val_full = get_preprocessed_dataset(data_path, months, sets, type, num_APs, scaling_strategy, floor)
df_val_x = df_val_full.iloc[:, :-3] # Just the RSSI values
df_val_y = df_val_full.iloc[:, -3:-1] # Just the x and y coordinates (no floor)

if DEBUG: print('df_val_full:', df_val_full.shape)

df_val_full: (864, 623)


In [83]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Since the implementations will be made in PyTorch, we convert the data to PyTorch tensors
X_train_tensor = torch.tensor(df_train_x.values, dtype=torch.float32)
y_train_tensor = torch.tensor(df_train_y.values, dtype=torch.float32)
X_test_tensor = torch.tensor(df_test_x.values, dtype=torch.float32)
y_test_tensor = torch.tensor(df_test_y.values, dtype=torch.float32)
X_val_tensor = torch.tensor(df_val_x.values, dtype=torch.float32)
y_val_tensor = torch.tensor(df_val_y.values, dtype=torch.float32)

# Get the data via DataLoaders
t_training = TensorDataset(X_train_tensor, y_train_tensor)
t_test = TensorDataset(X_test_tensor, y_test_tensor)
t_val = TensorDataset(X_val_tensor, y_val_tensor)

# train_loader = DataLoader(t_training, batch_size=16, shuffle=True)
# test_loader = DataLoader(t_test, batch_size=16, shuffle=True)
# val_loader = DataLoader(t_val, batch_size=16, shuffle=True)



# Full-Input Networks
These networks take the full input of 620 features to perform x,y predictions.

#### MultiLayer Perceptrons

In [84]:
SEARCH_MLP_FULL = True

In [85]:
import torch.nn as nn

class MLPfull(nn.Module):
    def __init__(self, hidden_layer_sizes, dropout_rate):
        super(MLPfull, self).__init__()
        
        layers = []
        input_dim = 620
        
        # Make it easier to grid-search different sizes of hidden layers
        for hidden_dim in hidden_layer_sizes:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            input_dim = hidden_dim # Ensure all hidden layers are constant size
        
        # At this point we know input_dim equals the output size of the last hidden layer, so we can re-use it here.
        layers.append(nn.Linear(input_dim, 2)) # x,y output
        
        # Construct the actual model based on the layers defined above.
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

In [86]:
def train_MLP_full(model, train_loader, val_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train() # Sets the model to training mode
        running_loss = 0.0 # Keep track of the (MSE) loss
        
        # Actual training loop
        for inputs, labels in train_loader:
            optimizer.zero_grad() # Reset gradients from last iteration
            outputs = model(inputs) # Forward pass
            loss = criterion(outputs, labels) # Compute the loss (MSE) between the predictions and the ground-truth labels
            loss.backward() # Perform backpropagation
            optimizer.step() # Update model parameters (weights) based on the gradients computed during backpropagation
            running_loss += loss.item() # Running loss is the sum of the losses for all batches FOR THE CURRENT EPOCH <-- TODO: (Make list for final model to plot)
        
        # Validation time
        model.eval()
        val_loss = 0.0 # Accumulated validation loss
        
        # Validation loop
        with torch.no_grad(): # No need to compute gradients during validation
            for inputs, labels in val_loader:
                outputs = model(inputs) # Forward pass to get predictions
                loss = criterion(outputs, labels) # Compute the loss (MSE) between the predictions and the ground-truth labels
                val_loss += loss.item() # Accumulate the validation loss for this epoch <-- TODO: (Make list for final model to plot)
        
        # Print the loss for this epoch
        print(f'Epoch {epoch+1}/{epochs} - Avg Training Loss: {running_loss/len(train_loader)} - Avg Validation Loss: {val_loss/len(val_loader)}')
    
    print('Finished Training')
    return val_loss/len(val_loader) # Return the average validation loss for final epoch

In [87]:
import optuna

def MLP_full_gridsearch(trial):
    # Hyper-parameters to be optimized
    
    # The line below does not work due to a optuna limitation. It is kept here for reference.
    #! hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [ (v,) * i for v in [700, 512, 256, 128] for i in range(2, 5)])
    
    hidden_layer_size = trial.suggest_categorical('hidden_layer_size', [700, 512, 256, 128])
    hidden_layer_count = trial.suggest_int('hidden_layer_count', 2, 4) # inclusive
    hidden_layer_sizes = (hidden_layer_size,) * hidden_layer_count
    
    dropout_rate = trial.suggest_float('dropout_rate', 0.4, 0.6)
    lr = trial.suggest_float('lr', 0.001, 0.01)
    batch_size = trial.suggest_int('batch_size', 16, 512, step=16)
    epochs = trial.suggest_int('epochs', 50, 150)
    
    # Initialize the model
    model = MLPfull(hidden_layer_sizes, dropout_rate)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) #TODO: Check other optimizer
    criterion = nn.MSELoss()
    
    # Use chosen batch size instead of pre-defined one
    train_loader = DataLoader(t_training, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(t_val, batch_size=batch_size, shuffle=True)
    
    # Train the model, return validation loss
    val_loss = train_MLP_full(model, train_loader, val_loader, criterion, optimizer, epochs)
    
    return val_loss

if SEARCH_MLP_FULL:
    print('Starting MLP full grid search')

    # Optuna study object and direction (minimize validation loss)
    study = optuna.create_study(direction='minimize')
    study.optimize(MLP_full_gridsearch, n_trials=25)

    print('====================================')
    print('Number of finished trials:', len(study.trials))
    print('Best trial:')
    trial = study.best_trial
    print('     Value: ', trial.value)
    print('     Params: ')
    for key, value in trial.params.items():
        print(f'         {key}: {value}')

else: print('Skipping MLP full grid search')

[I 2024-06-05 23:43:24,394] A new study created in memory with name: no-name-19746cf3-d649-4b1f-b404-e13673723da2


Starting MLP full grid search
Epoch 1/141 - Avg Training Loss: 461.39050242106117 - Avg Validation Loss: 22.01729170481364
Epoch 2/141 - Avg Training Loss: 45.4310119205051 - Avg Validation Loss: 33.11333988330983
Epoch 3/141 - Avg Training Loss: 35.277532725863985 - Avg Validation Loss: 90.47611377857349
Epoch 4/141 - Avg Training Loss: 25.320383008321127 - Avg Validation Loss: 66.91950988769531
Epoch 5/141 - Avg Training Loss: 17.478486442565917 - Avg Validation Loss: 28.29524591233995
Epoch 6/141 - Avg Training Loss: 11.964862166510688 - Avg Validation Loss: 13.938885653460467
Epoch 7/141 - Avg Training Loss: 10.634931786855061 - Avg Validation Loss: 10.84793703644364
Epoch 8/141 - Avg Training Loss: 9.95594023598565 - Avg Validation Loss: 10.150906280235008
Epoch 9/141 - Avg Training Loss: 9.162545511457655 - Avg Validation Loss: 7.224841541714138
Epoch 10/141 - Avg Training Loss: 9.530191527472603 - Avg Validation Loss: 8.00630467026322
Epoch 11/141 - Avg Training Loss: 8.96949489

[I 2024-06-05 23:44:10,884] Trial 0 finished with value: 9.527064535352919 and parameters: {'hidden_layer_size': 700, 'hidden_layer_count': 4, 'dropout_rate': 0.4049990911685759, 'lr': 0.009976921697034229, 'batch_size': 32, 'epochs': 141}. Best is trial 0 with value: 9.527064535352919.


Epoch 141/141 - Avg Training Loss: 11.679001596238878 - Avg Validation Loss: 9.527064535352919
Finished Training
Epoch 1/87 - Avg Training Loss: 161.10828590393066 - Avg Validation Loss: 41.00327014923096
Epoch 2/87 - Avg Training Loss: 40.67371400197347 - Avg Validation Loss: 40.88878345489502
Epoch 3/87 - Avg Training Loss: 29.235767364501953 - Avg Validation Loss: 11.918127536773682
Epoch 4/87 - Avg Training Loss: 25.303975741068523 - Avg Validation Loss: 24.144399642944336
Epoch 5/87 - Avg Training Loss: 22.083481788635254 - Avg Validation Loss: 7.736772537231445
Epoch 6/87 - Avg Training Loss: 19.364613850911457 - Avg Validation Loss: 12.721908330917358
Epoch 7/87 - Avg Training Loss: 17.003052552541096 - Avg Validation Loss: 7.72745144367218
Epoch 8/87 - Avg Training Loss: 15.926359494527182 - Avg Validation Loss: 9.806276559829712
Epoch 9/87 - Avg Training Loss: 15.330939292907715 - Avg Validation Loss: 9.051262140274048
Epoch 10/87 - Avg Training Loss: 13.962060133616129 - Avg 

[I 2024-06-05 23:44:13,890] Trial 1 finished with value: 8.187567234039307 and parameters: {'hidden_layer_size': 256, 'hidden_layer_count': 2, 'dropout_rate': 0.586563080922377, 'lr': 0.003472986511466822, 'batch_size': 240, 'epochs': 87}. Best is trial 1 with value: 8.187567234039307.


...

[I 2024-06-05 23:46:59,087] Trial 23 finished with value: 6.024723189848441 and parameters: {'hidden_layer_size': 512, 'hidden_layer_count': 3, 'dropout_rate': 0.5055455528319611, 'lr': 0.0032376231749212565, 'batch_size': 16, 'epochs': 50}. Best is trial 14 with value: 3.9730366865793862.


Epoch 50/50 - Avg Training Loss: 7.73105460802714 - Avg Validation Loss: 6.024723189848441
Finished Training
Epoch 1/77 - Avg Training Loss: 66.13769619281476 - Avg Validation Loss: 14.404905438423157
Epoch 2/77 - Avg Training Loss: 20.29773858877329 - Avg Validation Loss: 23.556578874588013
Epoch 3/77 - Avg Training Loss: 16.19034591087928 - Avg Validation Loss: 5.810279846191406
Epoch 4/77 - Avg Training Loss: 16.30988568526048 - Avg Validation Loss: 14.23047137260437
Epoch 5/77 - Avg Training Loss: 15.566931284390963 - Avg Validation Loss: 8.812962651252747
Epoch 6/77 - Avg Training Loss: 13.124860396751991 - Avg Validation Loss: 4.590393573045731
Epoch 7/77 - Avg Training Loss: 12.986891526442308 - Avg Validation Loss: 5.29109525680542
Epoch 8/77 - Avg Training Loss: 13.495770307687613 - Avg Validation Loss: 4.602854669094086
Epoch 9/77 - Avg Training Loss: 13.336853687579815 - Avg Validation Loss: 5.899704039096832
Epoch 10/77 - Avg Training Loss: 11.99879477574275 - Avg Validatio

[I 2024-06-05 23:47:06,101] Trial 24 finished with value: 4.146549701690674 and parameters: {'hidden_layer_size': 512, 'hidden_layer_count': 3, 'dropout_rate': 0.5309210545246215, 'lr': 0.0043503287722308015, 'batch_size': 112, 'epochs': 77}. Best is trial 14 with value: 3.9730366865793862.


Epoch 75/77 - Avg Training Loss: 6.97603284395658 - Avg Validation Loss: 5.4356513023376465
Epoch 76/77 - Avg Training Loss: 6.912521142225999 - Avg Validation Loss: 4.7036696672439575
Epoch 77/77 - Avg Training Loss: 6.758507691896879 - Avg Validation Loss: 4.146549701690674
Finished Training
Number of finished trials: 25
Best trial:
     Value:  3.9730366865793862
     Params: 
         hidden_layer_size: 128
         hidden_layer_count: 3
         dropout_rate: 0.5077424177283034
         lr: 0.00213362462464209
         batch_size: 16
         epochs: 65


: 